In [25]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\GAMING\AppData\Local\Temp\ipykernel_34148\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [26]:
pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\GAMING\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [28]:
data = pd.read_csv("C:/Users/GAMING/Desktop/ironahack/week 4/day 1/lap1/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')


In [29]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [ ]:
# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [30]:
# Check the columns in the DataFrame
print(data.columns)

Index(['text', 'label'], dtype='object')


### Let's divide the training and test set into two partitions

In [57]:
# Your code

from sklearn.model_selection import train_test_split

# Assuming 'text' is the feature and 'label' is the target variable
X = data['text']  # Features
y = data['label']    # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the shape of the training and test sets
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (4771,)
Test set shape: (1193,)


## Data Preprocessing

In [32]:
import nltk

# Download the stopwords dataset
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [34]:
# Your code
from bs4 import BeautifulSoup
import re

def clean_html(text):
    # Remove inline JavaScript and CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Use BeautifulSoup to remove remaining HTML tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    
    return text.strip()





In [35]:
# after step (clean)

# Clean the text data in the training and test sets
X_train_cleaned_html = X_train.apply(clean_html)
X_test_cleaned_html = X_test.apply(clean_html)

# Check the cleaned text
print(X_train_cleaned_html.head())

C:\Users\GAMING\AppData\Local\Temp\ipykernel_34148\3082045974.py:14: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, 'html.parser')


3821           back -- we need to talkCall only my berry.
2407                         What is mullen's first name?
1173    Greetings from Dubai=2CThis letter must come t...
4231    Sullivan Jacob J Friday December 4 2009 3:31 A...
1666                                                  Fyi
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [36]:
# Your code
import re

def clean_text(text):
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^\s*\w\s*', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b' (if it's part of a byte string representation)
    text = text.replace("b'", "").replace("'", "")
    
    # Convert to lowercase
    text = text.lower()
    
    return text




In [37]:
# after step (clean)
# Clean the text data in training and test sets
X_train_final_cleaned = X_train_cleaned_html.apply(clean_text)
X_test_final_cleaned = X_test_cleaned_html.apply(clean_text)

# Check the final cleaned text
print(X_train_final_cleaned.head())

3821                ack we need to talkcall only my berry
2407                            hat is mullens first name
1173    reetings from dubaicthis letter must come to y...
4231    ullivan jacob friday december amiranthe eu mee...
1666                                                   yi
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [38]:
# Download the stopwords dataset if you haven't already
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# Your code

from nltk.corpus import stopwords

# Get the set of English stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # Split the text into words and filter out stopwords
    filtered_text = ' '.join(word for word in text.split() if word not in stop_words)
    return filtered_text

In [40]:
# after step (clean)

# Remove stopwords from the cleaned text data
X_train_final_no_stopwords = X_train_final_cleaned.apply(remove_stopwords)
X_test_final_no_stopwords = X_test_final_cleaned.apply(remove_stopwords)

# Check the final text without stopwords
print(X_train_final_no_stopwords.head())

3821                              ack need talkcall berry
2407                               hat mullens first name
1173    reetings dubaicthis letter must come big surpr...
4231    ullivan jacob friday december amiranthe eu mee...
1666                                                   yi
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [41]:

# Download the WordNet resource if you haven't already
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
# Your code

from nltk.stem import WordNetLemmatizer

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Split the text into words
    words = text.split()
    
    # Lemmatize each word and join them back into a string
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in words)
    
    return lemmatized_text


In [43]:

# Apply lemmatization to the text data without stopwords
X_train_final_lemmatized = X_train_final_no_stopwords.apply(lemmatize_text)
X_test_final_lemmatized = X_test_final_no_stopwords.apply(lemmatize_text)

# Check the final lemmatized text
print(X_train_final_lemmatized.head())

3821                              ack need talkcall berry
2407                               hat mullens first name
1173    reetings dubaicthis letter must come big surpr...
4231    ullivan jacob friday december amiranthe eu mee...
1666                                                   yi
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code

import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

# Download necessary resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_and_tokenize(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return words

# Load data and process text
data_train = pd.read_csv(r'C:\Users\GAMING\Desktop\ironahack\week 4\day 1\lap1\lab-natural-language-processing\data\kg_train.csv').fillna('')
data_test = pd.read_csv(r'C:\Users\GAMING\Desktop\ironahack\week 4\day 1\lap1\lab-natural-language-processing\data\kg_test.csv').fillna('')

ham_train = data_train[data_train['label'] == 'ham']['text'].apply(clean_and_tokenize)
spam_train = data_train[data_train['label'] == 'spam']['text'].apply(clean_and_tokenize)

ham_words = [word for msg in ham_train for word in msg]
spam_words = [word for msg in spam_train for word in msg]

# Count frequencies and get top 10 most common words
top_10_ham = Counter(ham_words).most_common(10)
top_10_spam = Counter(spam_words).most_common(10)

def plot_top_words(top_words, title):
    if top_words:
        words, frequencies = zip(*top_words)
        plt.figure(figsize=(10, 5))
        plt.barh(words, frequencies, color='skyblue')
        plt.xlabel('Frequency')
        plt.title(title)
        plt.gca().invert_yaxis()
        plt.show()

plot_top_words(top_10_ham, 'Top 10 Words in Ham Messages (Train)')
plot_top_words(top_10_spam, 'Top 10 Words in Spam Messages (Train)')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GAMING\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Extra features

In [61]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,money_mark,suspicious_words,text_len
0,This is a test with euro and dollar.,1,0,36
1,Another example with free money and transfer.,1,1,45


## How would work the Bag of Words with Count Vectorizer concept?

In [62]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# Sample documents (corpus)
documents = [
    "I love programming",
    "Programming is fun",
    "I love fun programming"
]

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the documents into a matrix
X = vectorizer.fit_transform(documents)

# Convert to a dense array to view the result
print("Vocabulary:", vectorizer.get_feature_names_out())
print("Bag of Words Representation:\n", X.toarray())


Vocabulary: ['fun' 'is' 'love' 'programming']
Bag of Words Representation:
 [[0 0 1 1]
 [1 1 0 1]
 [1 0 1 1]]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [63]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample data
documents = [
    "I love programming",
    "Programming is fun",
    "I love fun programming"
]

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the documents into a TF-IDF matrix
X = vectorizer.fit_transform(documents)

# Print the shape of the TF-IDF matrix (documents x features)
print("Shape of TF-IDF matrix:", X.shape)

# If you want to view the actual TF-IDF values as an array
print("TF-IDF Matrix:\n", X.toarray())

# Vocabulary of terms used in the TF-IDF matrix
print("Vocabulary:", vectorizer.get_feature_names_out())


Shape of TF-IDF matrix: (3, 4)
TF-IDF Matrix:
 [[0.         0.         0.78980693 0.61335554]
 [0.54783215 0.72033345 0.         0.42544054]
 [0.61980538 0.         0.61980538 0.48133417]]
Vocabulary: ['fun' 'is' 'love' 'programming']


## And the Train a Classifier?

In [69]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Clean the text
data_train['clean_text'] = data_train['text'].apply(clean_and_tokenize)

# Split the data
X = data_train['clean_text']
y = data_train['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorization
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train the classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

# Make predictions
y_pred = classifier.predict(X_test_vectorized)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

Accuracy: 0.95
Confusion Matrix:
[[631  43]
 [ 18 501]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       674
           1       0.92      0.97      0.94       519

    accuracy                           0.95      1193
   macro avg       0.95      0.95      0.95      1193
weighted avg       0.95      0.95      0.95      1193



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code
